In [2]:
import numpy as np
import cv2
import os
# from PIL import Image
import matplotlib.pyplot as plt
import csv

Турецкие строчные буквы - abcçdefgğhiıjklmnoöprsştüvyz 

In [3]:
im = cv2.imread('./input/byron.png', cv2.IMREAD_GRAYSCALE)
cv2.imwrite('./input/gray_byron_orig.bmp', im)

True

In [32]:
class Segmentation:
    def __init__(self, input_array, output_path):
        self.data = {}
        self.output_path = output_path
        self.load_img(input_array, True)

    def load_img(self, path, is_folder):
        if is_folder:
            for filename in os.listdir(path):
                if filename.endswith(('.png', '.bmp')):
                    image_path = os.path.join(path, filename)
                    self.data[filename] = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE) 
        else:
            self.data[os.path.basename(path)] = cv2.imread(path,cv2.IMREAD_GRAYSCALE)

    def save_profiles_hist(self, hist, name):
        x = list(range(hist.size))
        plt.bar(x,hist,width=1.0)
        filename = os.path.join(self.output_path, name+".png")
        plt.savefig(filename)
        plt.clf()

    def get_segments(self, image, name):
        _, img = cv2.threshold(image, 150, 1, cv2.THRESH_BINARY_INV)
        H, W = img.shape[0], img.shape[1]
        print(H,W)

        H_profile = np.sum(img, axis=1)
        V_profile = np.sum(img, axis=0)

        self.save_profiles_hist(H_profile,name[:-4]+"_H_profile")
        self.save_profiles_hist(V_profile,name[:-4]+"_V_profile")

        left = None
        sym = []
        for i in range(len(V_profile)):
            if V_profile[i] > 0 and left is None:
                left = i
            elif V_profile[i] == 0 and left is not None:
                sym.append((left, i))
                left = None
        if left is not None:
            sym.append((left, len(V_profile)))

        boxes = []
        for s in sym:
            boxes.append((s[0], 0, s[1], H))
        
        return boxes
        
    def draw_segments(self):
        for name, img in self.data.items():
            boxes = self.get_segments(img,name)
            rgb_img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
            for b in boxes:
                cv2.rectangle(rgb_img,(b[0],b[1]),(b[2],b[3]),(0,255,0),1)
            filename = os.path.join(self.output_path, name)
            cv2.imwrite(filename, rgb_img)

In [33]:
agent = Segmentation("./input", "./output")
im = cv2.imread('./input/gray_byron.bmp', cv2.IMREAD_GRAYSCALE)
# boxes = agent.get_segments(im, 'gray_byron.bmp') 
# np.asarray(boxes).shape
agent.draw_segments()

49 1413


<Figure size 432x288 with 0 Axes>